# CS1470/2470 HW2: Multi-Layered Neural Networks

In this homework assignment, you will build a sequential model using differential modules.

---

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport Beras, assignment, preprocess

### Pull In The Data

In [2]:
# ![ ! -d '../../data' ] && bash cd ../.. && bash download.sh
data_path = "../data"

## Import MNIST train and test examples into train and testing data
from preprocess import load_and_preprocess_data

## Read in MNIST data,
X0, Y0, X1, Y1 = load_and_preprocess_data()

print("Training Shapes:", X0.shape, Y0.shape)
print("Testing  Shapes:", X1.shape, Y1.shape)

Training Shapes: (60000, 784) (60000,)
Testing  Shapes: (10000, 784) (10000,)


**> Expected Output** (double-click)
<!-- 
```
Training Shapes: (60000, 784) (60000,)
Testing  Shapes: (10000, 784) (10000,)
```

## Starting Our Modular API

### **The goal of this assignment is as follows:** 
- Extend our knowledge of deep learning by extending our single-layer model intuitions into multi-layer extensions. 
- Get familiarized with a simple modular API which reflects (but is a simplification of) the Keras analogue. 
    - Specifically, the `SequentialModel`
- Implement some nice modular components and be able to construct a functional single-file neural network from it. 

### **Exploring a possible modular implementation: TensorFlow/Keras**

We can check out what an established deep learning framework does to help us motivate our model. You'll learn more about this in the TensorFlow lab, but you can define a deep learning architecture using, among many other options, the `SequentialModel`:

```python 
from tensorflow.keras.optimizers import Adam                  ## Special
from tensorflow.keras.layers import Dense, LeakyReLU, Softmax ## Differentiable
from tensorflow.keras.losses import MeanSquaredError          ## Differentiable
from tensorflow.keras.metrics import MeanSquaredError         ## Non-Differentiable (but Callable)
from tensorflow.keras import Sequential                       ## Differentiable (surprisingly enough)

## Simple Model Architecture
tf.keras.Sequential([
    Dense(32),    ## ? -> 32 units per entry
    LeakyReLU(),  ## Bind logits to [min(l)*alpha, max(l)]
    Dense(1),     ## 4 -> 1 units per entry
    Softmax()     ## Bind logits to [0, 1] such that sum(logits) = 1
])

model.compile(
    optimizer   = Adam(learning_rate=1),
    loss        = MeanSquaredError(),
    metrics     = [MeanSquaredError()]
)

model.fit(X0, Y0, batch_size = 20, epochs = 10)
model.evaluate(X0, Y0, batch_size = 100)
```

This shows the main Keras value proposition: Modular components which you can move around and customize to your heart's content. This allows for rapid prototyping and small code bases, which allows almost anybody to get started with deep learning!

Keras itself is made up of building blocks from (and is a well-defined part of) Tensorflow. 
- **Keras:** Gives you high-level building blocks to keep track of internal components and train models easily.
- **Tensorflow**: Gives you the lower-level building blocks to easily differentiate and perform math operations. 

You'll find out more about this in the lab, but you can get a feel for how Tensorflow can be used by digging into the more low-level implementation of the Sequential Model *(which still uses Keras... I mean, we're not barbarians, right?)*: 

```python
import tensorflow as tf

class SequentialModel(tf.keras.Model):
    '''
    Note, this is a simplification, but it's close enough...
    It's also not exactly what you'll be doing (but really close)
    Note that a lot of things are missing: That's because they're handled by tf.keras.Model
    '''
    def __init__(self, layers):
        ## Phase at which you should specify the variables needed to do passes
        self.layers = layers

    def call(self, inputs, training=True):
        ## Phase at which you specify forward propagation
        x = tf.identity(inputs)   ## Copy input to de-reference it
        for layer in layers: 
            x = layer(x)
        return x

    def train_step(self, data):
        ## Optional Train_Step Specification
        ## This thing gets called for every batch
        x, y = data
        with tf.GradientTape() as tape:  ## While in scope of gradient tape: 
            ## Record what happens between all tf.Variable operations and 
            ## keep track of the partial gradients that get generated.
            logits  = self.call(x)          ## This is differentiable (i.e. implemented w/ tf.Variables)
            loss    = self.loss(y, logits)  ## This is also differentiable
        
        ## Figure out dL/dw for every trainable weights w based on 
        ## which operations were performed between them.
        grads = tape.gradient(loss, self.trainable_weights)
        optimizer.apply_gradients(zip(grads, self.trainable_weights))

        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, logits)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}
```

So... make it! ... kinda...

...yeah, that's a tall order. A few things to not about this:
- Tensorflow uses tf.Variables to do auto-differentiation from the operator level onward, which includes support for basic processes like addition and quotients and a bunch of other things. That's way too much detail for us to implement! 
- Real models oftentimes have variables and representations that diverge paths before reconvening into loss evaluations. In order to handle this, you'd need to implement a graph structure. This would just distract us from the main focus.

The sequential model allows us to simplify both of these away: 
- Since we only use large modules (i.e. Dense, ReLU, Sigmoid, MeanSquaredError, etc.), we can implement differentiation on that level: This will prove to be surprisingly tractible – but not exactly trivial, hence the assignment.
- Since the sequential model by default assumes a one-module-at-a-time pass-through, we don't have to worry too much about implicitly supporting primitive operations that cause sophisticated branching. 
    - Most components will only require one pathway in back-propagation: towards the partial with respect to its inputs. 
    - The dense layer will require the most amount of branching among our (1470) components, since it requires partials with respect to inputs, weights, and biases. We can handle that, right...?
    
Also note that the point of this exercise is not to implement a real library, nor is it to implement TF/Keras faithfully. As such, there is little consideration for realistic optimization (aside from vectorization) or scale beyond the assignment.

A more realistic (though still very pedagogical) implementation named [Brunoflow](https://github.com/Brown-Deep-Learning/brunoflow) is available from our very own Daniel Ritchie! It is a really nice library, but unfortunately is a bit too much for students to implement (and is also publicly available). Feel free to look at it whenever you want, as the two versions are very different. 

### Testing out our first Callable module: **OneHotEncoder**

To get you all warmed up to the idea of the modules, we have already provided you with an implementation of the `OneHotEncoder`.

- **`OneHotEncoder`**
    - **Pre-processing step:** Takes label dataset and converts each label $\ell \in L$ such that $\text{ohe}(\ell) = \mathbb{1}_{i=0}^{\|L\|}(i = \ell)$
    - **Callable:** Does not need to be differentiated, since not in optimization loop.
    - **Stateful:** Has to keep track of some states. For example, the set of unique elements and their mappings to the one-hot vectors.


In [4]:
%aimport Beras

from abc import ABC, abstractmethod  ## For abstract method support
import numpy as np

class Callable:
    
    def __call__(self, *args, **kwargs):
        '''Lets `self()` and `self.call()` be the same'''
        return self.call(*args, **kwargs)

    @abstractmethod
    def call(self, *args, **kwargs):
        """Propagates input through the network. Stores inputs and outputs as instance variables"""
        pass

class OneHotEncoder(Callable):
    '''
    One-Hot Encodes labels. First takes in a fit-set to figure out what elements it 
    needs to consider, and then one-hot encodes subsequent input datasets in the 
    forward pass (call). 
    
    SIMPLIFICATIONS: 
     - Implementation assumes that entries are individual elements.
     - keras does not have OneHotEncoder; has LabelEncoder, CategoricalEncoder, and to_categorical()
    '''  
    def fit(self, data):
        '''
        Fits the one-hot encoder to a candidate dataset. Said dataset should contain 
        all encounterable elements.
        '''
        self.uniq = np.unique(data)
        self.vecs = np.eye(len(self.uniq))
        self.uniq2oh = {e : self.vecs[i] for i, e in enumerate(self.uniq)} 

    def call(self, data):
        if not hasattr(self, 'uniq2oh'): self.fit(data)
        return np.array([self.uniq2oh[x] for x in data])

    def inverse(self, data):
        ## Special custom method useful for OHE. Not generally expected in other instances
        assert hasattr(self, 'uniq'), 'call() or fit() must be called before attempting to invert'
        return np.array([self.uniq[x == 1][0] for x in data])

ohe = OneHotEncoder()
ohe.fit(np.concatenate([Y0, Y1], axis=-1))

print("Getting label sample:")
print(Y0[:10])
print("Testing Call:")
print(''.join([f'  {i}' for i in range(10)]), 'is hot')
print(ohe(Y0[:10]))
print("Testing Inverse:")
print(ohe.inverse(ohe(Y0[:10])))

UsageError: Line magic function `%aimport` not found.


### Testing out our first Diffable module: **Categorical Cross-Entropy**

An extension of Callable – a module that can be called – is the Diffable – a module that can be Diff(erentiat)ed. The details are quite... specific. We (or at least Vadim) originally wanted you all to do this, but then we realized that this would take an extremely long amount of time to implement and debug even if you already knew all of the concepts. So instead, we've provided the source code for this whole part (in fact, almost all of `Beras/core.py`) in the assignment stencil. Sometimes it's better to have you struggle to reason about a well-debugged implementation than to actually have you implement it yourself... or at least that's the mindset we're going with. Hope you appreciate it :)

```python
class Diffable(ABC, Callable):

    '''Is the operation being used inside a gradient tape scope?'''
    gradient_tape = None    ## All-instance-shared variable

    def __call__(self, *args, **kwargs):
        '''
        If there is a gradient tape scope in effect, perform AND RECORD the operation.
        Otherwise... just perform the operation and don't let the gradient tape know. 
        '''

    @abstractmethod
    def input_gradients(self):
        """Returns gradient for input (this part gets specified for all diffables)"""

    def weight_gradients(self):
        """Returns gradient for input (this part gets specified for SOME diffables)"""

    def compose_to_input(self, J):
        """
        Compose the inputted cumulative jacobian with the input jacobian for the layer.
        Implemented with batch-level vectorization.

        Requires `input_gradients` to provide either batched or overall jacobian.
        Assumes input/cumulative jacobians are matrix multiplied
        """

    def compose_to_weight(self, J):
        """
        Compose the inputted cumulative jacobian with the weight jacobian for the layer.
        Implemented with batch-level vectorization.

        Requires `weight_gradients` to provide either batched or overall jacobian.
        Assumes weight/cumulative jacobians are element-wise multiplied (w/ broadcasting)
        and the resulting per-batch statistics are averaged together for avg per-param gradient.
        """
```

#### **Simple Diffable Example:** Sin Activation

In [ ]:
%aimport Beras

from Beras.core import Diffable
import numpy as np

class Sin(Diffable):

    def call(self, inputs):
        return np.sin(inputs)

    def input_gradients(self):
        return [np.cos(self.inputs)]
        
    def compose_to_input(self, J):
        ## One of the few times you may want to do this.
        ## We'll leave it up to you to figure out when.
        return self.input_gradients()[0] * J

act_fn = Sin()
## 2 batches with 3 entries and 4 elements per entry
sample = np.arange(-12, 12).reshape(2, 3, 4)       
out = act_fn(sample)

print("Activation Input:")
print(sample)

print("\nActivation Output:")
print(out)

print("\nInput Gradients:")
print(act_fn.input_gradients())

print("\nCompose To Input:")
print(act_fn.compose_to_input(out))

print("\nSanity Check:")
comp1 = act_fn.compose_to_input(act_fn.input_gradients())[0]
comp2 = act_fn.input_gradients()[0] * act_fn.input_gradients()[0]
if np.allclose(comp1, comp2):
    print("The world still makes sense")

#### **Advanced Diffable Example:** MeanSquaredError

In [ ]:
%aimport Beras

from Beras.core import Diffable
import numpy as np

class MeanSquaredError(Diffable):

    def __init__(self, *args, from_logits=True, **kwargs):
        ## We're redirecting all of our arguments but also supporting an extra one!
        super().__init__(*args, **kwargs)
        self.from_logits = from_logits

    def call(self, y_pred, y_true):
        """Mean squared error forward pass!"""
        if self.from_logits:
            mse_total = np.mean(np.power(y_pred - y_true, 2), axis=-1)
        else: 
            mse_total = np.mean(np.power(y_pred * y_true - y_true, 2), axis=-1)
        return np.mean(mse_total, axis=0)

    def input_gradients(self):
        """Mean squared error input gradient!"""
        y_pred, y_true = self.inputs
        if self.from_logits:
            ## Using all entries. Sub-par if using softmax
            grad = 2 * (y_pred - y_true) / np.prod(y_pred.shape)
        else:
            ## Only using target class. Better for softmax
            grad = 2 * ((y_pred * y_true) - y_true) / y_pred.shape[0]
        return [grad, -grad]
    
loss_fn = MeanSquaredError()
## 3 batches with 3 elements per batch (losses, right?)
ones = np.ones((3, 3)) 
zeros = np.zeros_like(ones)

print("Awful performance")
print(loss_fn(ones, zeros))
print(loss_fn(zeros, ones))
print("Input Gradients:")
print(loss_fn.input_gradients())

print("\nPerfect performance")
print(loss_fn(ones, ones))
print(loss_fn(zeros, zeros))
print("Input Gradients:")
print(loss_fn.input_gradients())

print("\nNot great performance")
print(loss_fn(ones * 0.5, ones))
print("Input Gradients:")
print(loss_fn.input_gradients())

print("\nWeight Gradients:")
print(loss_fn.weight_gradients())

print("\nCompose To Input:")
print(loss_fn.compose_to_input())

print("\nSanity Check:")
comp1 = loss_fn.compose_to_input(loss_fn.input_gradients())[0]
comp2 = loss_fn.input_gradients()[0] @ loss_fn.input_gradients()[0]
if np.allclose(comp1, comp2):
    print("The world still makes sense")